In [1]:
################################################################################
# Remove columns with mean 0 and remove duplicate rows - Train
################################################################################

import numpy as np
import os
import pandas as pd
from tqdm import tqdm

df_train = pd.read_csv(os.getcwd() + '/../data/bot-iot/Bot-IoT_Dataset/Dataset/5%/10-best features/10-best Training-Testing split/UNSW_2018_IoT_Botnet_Final_10_best_Training.csv', low_memory=False)

# remove columns if mean is 0
columns_to_drop = []
for c in df_train.select_dtypes(include=[np.number]).columns:
    if df_train[c].mean() == 0:
        columns_to_drop.append(c)
print("Columns to drop: ", columns_to_drop)

# get df with removed columns
df_train_with_dropped_columns = df_train.drop(columns_to_drop, axis=1)
print("Number of columns before removing columns: ", len(df_train.columns))
print("Number of columns after removing columns: ", len(df_train_with_dropped_columns.columns))

# remove duplicate rows
df_train_no_duplicates = df_train_with_dropped_columns.drop_duplicates()
print("Length of dataset before removing duplicates: ", len(df_train_with_dropped_columns))
print("Length of dataset after removing duplicates: ", len(df_train_no_duplicates))

# save to csv
df_train_no_duplicates.to_csv(os.getcwd() + '/data/population_train.csv', index=False)


Columns to drop:  []
Number of columns before removing columns:  19
Number of columns after removing columns:  19
Length of dataset before removing duplicates:  2934817
Length of dataset after removing duplicates:  2934817


In [2]:
################################################################################
# Remove columns with mean 0 and remove duplicate rows - Test
################################################################################

import numpy as np
import os
import pandas as pd
from tqdm import tqdm

df_test = pd.read_csv(os.getcwd() + '/../data/bot-iot/Bot-IoT_Dataset/Dataset/5%/10-best features/10-best Training-Testing split/UNSW_2018_IoT_Botnet_Final_10_best_Testing.csv', low_memory=False)

# remove columns if mean is 0
columns_to_drop = []
for c in df_test.select_dtypes(include=[np.number]).columns:
    if df_test[c].mean() == 0:
        columns_to_drop.append(c)
print("Columns to drop: ", columns_to_drop)

# get df with removed columns
df_test_with_dropped_columns = df_test.drop(columns_to_drop, axis=1)
print("Number of columns before removing columns: ", len(df_test.columns))
print("Number of columns after removing columns: ", len(df_test_with_dropped_columns.columns))

# remove duplicate rows
df_test_no_duplicates = df_test_with_dropped_columns.drop_duplicates()
print("Length of dataset before removing duplicates: ", len(df_test_with_dropped_columns))
print("Length of dataset after removing duplicates: ", len(df_test_no_duplicates))

# save to csv
df_test_no_duplicates.to_csv(os.getcwd() + '/data/population_test.csv', index=False)


Columns to drop:  []
Number of columns before removing columns:  19
Number of columns after removing columns:  19
Length of dataset before removing duplicates:  733705
Length of dataset after removing duplicates:  733705
